<a href="https://colab.research.google.com/github/Bluedata-Consulting/GAAPB01-training-code-base/blob/main/SLEFLEARNING_Prompt_Engineering_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering Techniques



This guide provides an introduction to prompt engineering techniques for Azure OpenAI services, focusing on how to craft effective prompts to elicit desired responses from large language models (LLMs).

### Core Concepts

**What is Prompt Engineering?**

Prompt engineering is the art and science of designing effective prompts to guide LLMs toward generating the desired output. It's about understanding how LLMs process information and using that knowledge to create prompts that are clear, specific, and unambiguous.

**Why is Prompt Engineering Important?**

* **Accuracy:** Well-crafted prompts significantly improve the accuracy and relevance of the generated responses.
* **Control:**  Prompt engineering gives you control over the style, format, and content of the output.
* **Efficiency:** Effective prompts can minimize the number of iterations needed to achieve the desired results, saving time and resources.
* **Customization:** Prompt engineering allows you to tailor the LLM's behavior to specific tasks and domains.


### Key Techniques and Examples


**1.  Clear and Specific Instructions:**

* **Bad Prompt:**  "Tell me about dogs."  (Too vague, could result in a wide range of responses.)
* **Good Prompt:** "Write a 200-word essay comparing and contrasting the characteristics of Golden Retrievers and German Shepherds." (Specific, sets length and topic.)


**2.  Few-Shot Learning:**

Provide examples of input-output pairs to guide the model.

* **Example:**  


Make Sure, Below Environment Variables are created already:


```
AZURE_OPENAI_API_KEY
AZURE_OPENAI_ENDPOINT

```

In [2]:
import os
os.environ['AZURE_OPENAI_ENDPOINT'] = ""
os.environ['AZURE_OPENAI_API_KEY'] = ""

In [3]:
from openai import AzureOpenAI
client = AzureOpenAI(api_version="2024-12-01-preview")

In [4]:
# creating a function to get outcome
def generate_response(prompt,model="telcogpt",temperature=0):
  messages = [{"role":"user","content":prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=temperature
  )
  return response.choices[0].message.content


## Zero Shot Prompting

In [5]:
# zero shot prompting
prompt = f"""
Create a conversation story between child and grandparent.
"""
response = generate_response(prompt)
print(response)

**Child:** Grandma, can you tell me a story from when you were little?

**Grandparent:** Oh, I’d love to! When I was your age, we didn’t have computers or tablets. We played outside all day.

**Child:** Really? What did you play?

**Grandparent:** We played hide and seek, climbed trees, and made up our own games. One time, my friends and I built a little fort out of sticks and leaves.

**Child:** That sounds fun! Did you ever get scared?

**Grandparent:** Sometimes, especially when it got dark. But we always stuck together and helped each other. It made us brave.

**Child:** I wish I could play like that. I spend a lot of time on my tablet.

**Grandparent:** It’s okay to use your tablet, but don’t forget to play outside too. Nature has its own magic.

**Child:** I’ll try! Can we build a fort together sometime?

**Grandparent:** Absolutely! Next weekend, we’ll gather some sticks and make the best fort ever.

**Child:** Yay! I can’t wait! Thanks, Grandma.

**Grandparent:** You’re welcome

## Few Shot Prompting

Here, the model is given a few examples (shots) to guide its response. By providing context or previous instances, the model can better understand and generate the desired output. For example, showing a model several examples of translated sentences before asking it to translate a new one.


In [6]:
# few shot prompting
prompt = f"""
your task is to answer in a consistent sytle:

<child>: Teach me about patience.
<grandma>: Patience is like waiting for whole day to see the moon in the evening and then sleeping after having a look at it.

<child>: Teach me about resilience.
"""
response = generate_response(prompt)
print(response)

<grandma>: Resilience is like a tree that bends in the wind but doesn’t break, standing tall again when the storm has passed.


#### Few shot prompting with role setup

In [8]:
result = client.chat.completions.create(
    model='telcogpt',  # e.g. gpt-35-instant
    max_tokens=200,
    temperature=0.9,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "That was an awesome experience"},
        {"role": "assistant", "content": "positive"},
        {"role": "user", "content": "I won't do that again"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "That was not worth my time"},
        {"role": "assistant", "content": "negative"},
        {"role": "user", "content": "You can't miss this"}
    ],
)
print(result.choices[0].message.content)

positive


## Chain of Thought Prompting

In [9]:
# Without Chain of Thought Prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Let's break down the problem step-by-step:

- Marry initially had 5 pens.
- She gave 3 pens to her friend, so she has \(5 - 3 = 2\) pens left.
- She bought 2 boxes of pens, with 3 pens in each box, so she got \(2 \times 3 = 6\) pens.
- Adding these to the pens she has left: \(2 + 6 = 8\) pens.

**Answer:** Marry has 8 pens now.


In [10]:
# with Chain of Thought prompting
prompt = f"""

Teacher: Johan had 5 apples, he bought 4 more boxes with 5 apples each. How many apples does he have now?
Student: Johan had 5 apples, 4 boxes with 5 apples each = 5 + 4*5 = 25 apples

Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Marry had 5 pens, she gave 3 to her friend, so she has 5 - 3 = 2 pens left.

Then she bought 2 boxes with 3 pens in each box, so she got 2 * 3 = 6 pens.

Total pens now = 2 + 6 = 8 pens.

Answer: Marry has 8 pens now.


In [11]:
# with Chain of Thought prompting
prompt = f"""

Think step by step to answer below question, do not come to conclusion without following process.
Teacher: Marry had 5 pens, she gave 3 to her friend. and bought 2 boxes of pen with 3 pen in each box. How many pens does she have now?
Student:

"""
response = generate_response(prompt)
print(response)

Let's analyze the problem step by step:

1. **Initial number of pens Marry had:**  
   Marry had 5 pens.

2. **Pens given away:**  
   She gave 3 pens to her friend.  
   So, pens left = 5 - 3 = 2 pens.

3. **Pens bought:**  
   She bought 2 boxes of pens, with 3 pens in each box.  
   Total pens bought = 2 × 3 = 6 pens.

4. **Total pens now:**  
   Pens left after giving away + pens bought = 2 + 6 = 8 pens.

**Answer:** Marry now has 8 pens.


## Tree of Thoughts (ToT)

Tree of Thoughts (ToT) prompting is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem-solving with language models.


How does it work?

- ToT prompting breaks problems down into smaller parts, similar to CoT prompting, but goes further by combining this with the ability to explore multiple solution paths in parallel, forming a tree instead of a single chain. Each thought is generated or solved independently and passed to the next step, allowing the model to self-evaluate and decide whether to continue with that path or choose another.

In [12]:

prompt = """
Imagine 5 different experts are answering this question.
They will brainstorm the answer step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on science and the laws of physics
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought
Each expert will assign a likelihood of their current assertion being correct
Continue until the experts agree on the single most likely location
The question is.
1. Carlos is at the swimming pool.
2. He walks to the locker room, carrying a towel.
3. He puts his watch in the towel and carries the towel tightly to a lounger at the poolside.
4. At the lounger he opens and vigorously shakes the towel, then walks to the snack bar.
5. He leaves the towel at the snack bar, then walks to the diving board.
6. Later Carlos realises he has has lost his watch. Where is the single most likely location of the watch?
"""


ans=generate_response(prompt)

print(ans)

Let's begin the expert panel discussion. We have 5 experts: 

- Expert A: Physicist specializing in mechanics and motion
- Expert B: Cognitive psychologist specializing in human behavior and memory
- Expert C: Forensic investigator specializing in lost item recovery
- Expert D: Environmental scientist specializing in poolside environments
- Expert E: Security analyst specializing in surveillance and theft prevention

---

### Step 1: Each expert writes down their initial reasoning step.

---

**Expert A (Physicist):**  
Step 1: The watch was placed inside the towel, which Carlos carried tightly to the lounger. When Carlos shook the towel vigorously, the watch could have been dislodged and fallen out at that point. Given the laws of motion and gravity, the watch would most likely fall near the lounger, possibly on the ground or the lounger itself.

Likelihood of this assertion being correct: 70%

---

**Expert B (Psychologist):**  
Step 1: Carlos’s memory of the watch’s location is like

## Chain-of-Verification Prompting

The Chain-of-Verification (CoVe) prompt engineering method aims to reduce hallucinations through a verification loop.

In [13]:
Question="Name some athletes who were born in United states"

prompt=f'''Here is the question: {Question}.

First, generate a response.

Then, create and answer verification questions based on this response to check for accuracy. Think it through and make sure you are extremely accurate based on the question asked.

After answering each verification question, consider these answers and revise the initial response to formulate a final, verified answer. Ensure the final response reflects the accuracy and findings from the verification process.'''



ans=generate_response(prompt)

print(ans)


**Initial Response:**

Some athletes who were born in the United States include:

1. Michael Jordan (basketball)
2. Serena Williams (tennis)
3. Tom Brady (American football)
4. Simone Biles (gymnastics)
5. Tiger Woods (golf)

---

**Verification Questions and Answers:**

1. Was Michael Jordan born in the United States?  
   - Yes, Michael Jordan was born in Brooklyn, New York, USA.

2. Was Serena Williams born in the United States?  
   - Yes, Serena Williams was born in Saginaw, Michigan, USA.

3. Was Tom Brady born in the United States?  
   - Yes, Tom Brady was born in San Mateo, California, USA.

4. Was Simone Biles born in the United States?  
   - Yes, Simone Biles was born in Columbus, Ohio, USA.

5. Was Tiger Woods born in the United States?  
   - Yes, Tiger Woods was born in Cypress, California, USA.

---

**Final Verified Response:**

Some athletes who were born in the United States include:

1. Michael Jordan (basketball) – born in Brooklyn, New York  
2. Serena Williams (t

# Model Limitations: Hallucinations

In [14]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


Step 1: Get some warm water.  
Step 2: Mix the warm water with some detergent.  
Step 3: Put the dirty clothes into the water and detergent mixture.  
Step 4: Let the clothes soak for some time to loosen the stains.  
Step 5: Take the clothes out and rub the dirt or stains with a soft brush to remove any remaining stains.  
Step 6: Rinse the clothes with clean water to remove the detergent.  
Step 7: Hang the clothes on a rope to dry.  
Step 8: Enjoy your clean clothes.


In [15]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks. Rewrite those into a set of instructions in the following format:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```
"""
response = generate_response(prompt)
print(response)


Step 1: Observe the morning and notice the bright sun shining between the clouds.  
Step 2: Appreciate the pleasant weather around you.  
Step 3: Feel the refreshing and mindblowing wind.  
Step 4: Take a moment to enjoy and love the day.


In [16]:
# asking for a specific format
text = """
Washing clothes without washing machines is easy!
First you need to get some warm water. Mix the warm water with some detergent. Second you can put the dirty clothes in it and let it stay wet for some time.
During this period the stains will get loose and mixed with water.
thereafter you can take clothes out, rub the dirt/stains with a soft brush to remove the stains if any.
get the clothes mixed with water to get the detergent out of it. and then put it for drying on a rope.
Enjoy the clean clothes.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


Step 1: Get some warm water.  
Step 2: Mix the warm water with some detergent.  
Step 3: Put the dirty clothes in the mixture and let them stay wet for some time.  
Step 4: Take the clothes out and rub the dirt or stains with a soft brush to remove them.  
Step 5: Rinse the clothes with water to get the detergent out.  
Step 6: Hang the clothes on a rope to dry.  
Step 7: Enjoy the clean clothes.


In [17]:
# asking for a specific format
text = """
THe morning today is amazing, the sun is looking bright in between clouds, the weather is pleasent and the wind is mindblowing. on a whole i am loving the day today.
"""

prompt = f"""
you will be provided with some text delimited by triple backticks.
Rewrite those into a set of instructions in the following format only if contains a sequence of instructions:

Step 1: ...
Step 2: ....
..
Step N:...

```{text}
```

if the text does not contain a sequence of instructions, then simply write 'No steps provided'
"""
response = generate_response(prompt)
print(response)


No steps provided


In [18]:
prompt = f"""
Tell me about Ergonamic chair procuced and sold by Morning Star IT Services.
"""

generate_response(prompt)

"I couldn't find specific information about an ergonomic chair produced and sold by Morning Star IT Services. It's possible that this product is new, niche, or not widely documented online.\n\nIf you have any additional details or context about the chair, such as a model name or features, please share them, and I can help you better. Alternatively, you might want to check Morning Star IT Services' official website or contact them directly for the most accurate and detailed information about their ergonomic chair."

In [19]:
prompt = f"""
Tell me about Ergonamic Toothbrush by colgate.
"""

generate_response(prompt)

"The **Ergonomic Toothbrush by Colgate** is designed to provide a comfortable and effective brushing experience by focusing on user-friendly design features. While specific models may vary, here are some common characteristics of Colgate's ergonomic toothbrushes:\n\n1. **Comfortable Handle Design**: The handle is typically shaped to fit comfortably in the hand, often featuring non-slip grips or rubberized sections to ensure better control and reduce hand fatigue during brushing.\n\n2. **Optimized Bristle Arrangement**: The bristles are arranged to reach different areas of the mouth effectively, including hard-to-reach spots, helping to remove plaque and promote better oral hygiene.\n\n3. **Flexible Neck**: Some ergonomic toothbrushes include a flexible neck that helps absorb excess pressure while brushing, protecting gums from damage.\n\n4. **Variety of Bristle Types**: Options may include soft, medium, or firm bristles, catering to different user preferences and dental needs.\n\n5. **

In [20]:
prompt = f"""
Tell me about Ultraslim Toothbrush by MorningStar.
"""

generate_response(prompt)

'The Ultraslim Toothbrush by MorningStar is designed to provide effective oral care with a sleek and slim profile. Key features typically include:\n\n- **Ultra-thin handle:** Designed for a comfortable grip and easy maneuverability, allowing you to reach all areas of your mouth.\n- **Soft bristles:** Gentle on gums while effectively cleaning teeth and removing plaque.\n- **Compact head:** Helps access hard-to-reach areas, promoting thorough cleaning.\n- **Durability:** Made with quality materials to ensure long-lasting use.\n\nMorningStar’s Ultraslim Toothbrush is often praised for combining functionality with a minimalist design, making it a popular choice for those seeking a simple yet effective toothbrush.\n\nIf you want specific details like pricing, availability, or user reviews, please let me know!'

In [21]:
prompt = f"""
Generate one line description for each of the following city:
1. Manila
2. Delhi
3. MorningStarCity
"""

print(generate_response(prompt))

1. Manila: The vibrant capital of the Philippines, known for its rich history, bustling streets, and cultural landmarks.  
2. Delhi: India's sprawling capital, blending ancient heritage with modern urban life and political significance.  
3. MorningStarCity: A futuristic metropolis renowned for its innovative technology, sustainable living, and dynamic skyline.


In [22]:
prompt = f"""
Generate one line description for each of the following city, check if the city exists, if it does not exist, say no description can be generated:
1. Manila
2. Delhi
3. MorningStarCity
"""

print(generate_response(prompt))

1. Manila: The capital city of the Philippines, known for its rich history and vibrant urban culture.  
2. Delhi: The bustling capital territory of India, famous for its historic monuments and diverse culture.  
3. MorningStarCity: No description can be generated.


## Conversational App

In [25]:
# creating a function to get outcome
def generate_response(messages,model='telcogpt'):
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature=0.5
  )
  return response.choices[0].message.content


In [26]:
messages = [{"role":"system","content":"You are a helpful assistant"},]
while True:
    uinput = input("You: ")
    print("You: ",uinput)
    if uinput.lower() in ['q','quit','exit']:
        break
    messages.append({"role":"user","content":uinput})
    response = generate_response(messages=messages)
    print(f"Bot: {response}")
    messages.append({"role":"assistant","content":response})


You: hi
You:  hi
Bot: Hello! How can I assist you today?
You: exit()
You:  exit()
Bot: If you want to end our conversation, that's perfectly fine. Have a great day! If you need any help later, feel free to reach out. Goodbye!
You: exit
You:  exit


In [ ]:
messages

## Thank You